In [18]:
import json
code_mappings = json.load(open('/Users/ishekhtman/Documents/Apixio/hoisting table validation/resources/hcc-code-mappings.js'))
apixio_hcc_to_icd = 

for code_mapping in code_mappings:

    if code_mapping['fromCodingSystem'] == '2.16.840.1.113883.6.90': # if this is ICD-10
        hcc_key = code_mapping['labelSet'] + '-' + code_mapping['hcc'] # key value by labelSet and HCC
        codes = apixio_hcc_to_icd[hcc_key] if hcc_key in apixio_hcc_to_icd else {'icd9s':[],'icd10s':[]}
        codes['icd10s'].append(code_mapping['fromCode'])
        apixio_hcc_to_icd[hcc_key] = codes
    
    if code_mapping['fromCodingSystem'] == '2.16.840.1.113883.6.103': # if this is ICD-9
        hcc_key = code_mapping['labelSet'] + '-' + code_mapping['hcc'] # key value by labelSet and HCC
        codes = apixio_hcc_to_icd[hcc_key] if hcc_key in apixio_hcc_to_icd else {'icd9s':[],'icd10s':[]}
        codes['icd9s'].append(code_mapping['fromCode'])
        apixio_hcc_to_icd[hcc_key] = codes

In [19]:
import csv
cms_hcc_to_icd = {}

# ICD9 Codes
with open('/Users/ishekhtman/Documents/Apixio/hoisting table validation/code_mappings/icd9_mappings.csv', 'rb') as csvfile:
    hccreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for icd9_mapping in hccreader:
        icd9_code = icd9_mapping[0].strip()         # DIAGNOSIS CODE
        desc = icd9_mapping[1].strip()              # SHORT DESCRIPTION
        hcc_category_2013 = icd9_mapping[2].strip() # 2013 CMS-HCC Model Category
        hcc_category_pace = icd9_mapping[3].strip() # CMS-HCC PACE/ESRD Model Category
        hcc_category_2014 = icd9_mapping[4].strip() # 2014 CMS-HCC Model Category
        hcc_category_rx = icd9_mapping[5].strip()   # RxHCC Model Category
        hcc_model_2013 = icd9_mapping[6].strip()    # 2013 CMS-HCC Model for 2014 Payment Year
        hcc_model_pace = icd9_mapping[7].strip()    # CMS-HCC PACE/ESRD Model for 2014 Payment Year
        hcc_model_2014 = icd9_mapping[8].strip()    # 2014 CMS-HCC Model for 2014 Payment Year
        hcc_model_rx = icd9_mapping[9].strip()      # RxHCC Model for 2014 Payment Year

        hcc_keys = []
        if hcc_model_2013 == 'Yes':
            hcc_keys.append('V12-' + hcc_category_2013)
        if hcc_model_2014 == 'Yes':
            hcc_keys.append('V22-' + hcc_category_2014)

        for key in hcc_keys:
            mapping = cms_hcc_to_icd[key] if key in cms_hcc_to_icd else {'icd9s':set(),'icd10s':set()}
            mapping['icd9s'].add(icd9_code)
            cms_hcc_to_icd[key] = mapping

# ICD10 Codes
with open('/Users/ishekhtman/Documents/Apixio/hoisting table validation/code_mappings/icd10_mappings.csv', 'rb') as csvfile:
    hccreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for icd10_mapping in hccreader:
        icd10_code = icd10_mapping[0].strip()         # Diagnosis Code
        desc = icd10_mapping[1].strip()               # Description
        hcc_category_v21 = icd10_mapping[2].strip()   # CMS-HCC PACE/ESRD Model Category V21
        hcc_category_v22 = icd10_mapping[3].strip()   # CMS-HCC Model Category V22
        hcc_category_v05 = icd10_mapping[4].strip()   # RxHCC Model Category V05 (clinically revised model implemented in 2016)
        hcc_model_v21 = icd10_mapping[5].strip()      # CMS-HCC PACE/ESRD Model for 2016 Payment Year
        hcc_model_v22 = icd10_mapping[6].strip()      # CMS-HCC Model for 2016 Payment Year
        hcc_model_v05 = icd10_mapping[7].strip()      # RxHCC Model for 2016 Payment Year

        hcc_keys = []
        if hcc_model_v22 == 'Yes':
            hcc_keys.append('V22-' + hcc_category_v22)

        for key in hcc_keys:
            mapping = cms_hcc_to_icd[key] if key in cms_hcc_to_icd else {'icd9s':set(),'icd10s':set()}
            mapping['icd10s'].add(icd10_code)
            cms_hcc_to_icd[key] = mapping

In [24]:
# list the HCCs that are present in the CMS model but not in Apixio Code Mappings
print [hcc for hcc in cms_hcc_to_icd if hcc not in apixio_hcc_to_icd]
# list the HCCs that are present in the Apixio model but not in HCC model
print [hcc for hcc in apixio_hcc_to_icd if hcc not in cms_hcc_to_icd]

# Result: both Apixio and CMS have the same HCCS

print len(cms_hcc_to_icd)
print len(apixio_hcc_to_icd)


[]
[]
149
149


In [26]:
hcc_diff = {}
# check if Apixio and CMS agree on the ICD codes used for a particular HCC
for hcc in cms_hcc_to_icd:
    icd9s_not_in_apixio = [icd for icd in cms_hcc_to_icd[hcc]['icd9s'] if icd not in apixio_hcc_to_icd[hcc]['icd9s']]
    icd10s_not_in_apixio = [icd for icd in cms_hcc_to_icd[hcc]['icd10s'] if icd not in apixio_hcc_to_icd[hcc]['icd10s']]
    icd9s_not_in_cms = [icd for icd in apixio_hcc_to_icd[hcc]['icd9s'] if icd not in cms_hcc_to_icd[hcc]['icd9s']]
    icd10s_not_in_cms = [icd for icd in apixio_hcc_to_icd[hcc]['icd10s'] if icd not in cms_hcc_to_icd[hcc]['icd10s']]
    
    if len(icd9s_not_in_apixio) > 0 or len(icd10s_not_in_apixio) > 0 or len(icd9s_not_in_cms) > 0 or len(icd10s_not_in_cms) > 0:
        hcc_diff[hcc] = {'icd9s_not_in_apixio':icd9s_not_in_apixio,
                         'icd10s_not_in_apixio':icd10s_not_in_apixio,
                         'icd9s_not_in_cms':icd9s_not_in_cms,
                         'icd10s_not_in_cms':icd10s_not_in_cms}
from pprint import pprint
with open('hoisting_analysis_diff.json','w') as diff:
    pprint(hcc_diff,diff)
    pprint(hcc_diff)

{'V22-10': {'icd10s_not_in_apixio': ['C965',
                                     'C9030',
                                     'C9031',
                                     'C9032',
                                     'C966'],
            'icd10s_not_in_cms': [],
            'icd9s_not_in_apixio': [],
            'icd9s_not_in_cms': []},
 'V22-11': {'icd10s_not_in_apixio': [],
            'icd10s_not_in_cms': [u'C452'],
            'icd9s_not_in_apixio': [],
            'icd9s_not_in_cms': []},
 'V22-112': {'icd10s_not_in_apixio': [],
             'icd10s_not_in_cms': [u'D861',
                                   u'D863',
                                   u'D8681',
                                   u'D8682',
                                   u'D8683',
                                   u'D8684',
                                   u'D8685',
                                   u'D8686',
                                   u'D8687',
                                   u'D8689',
         

In [23]:
from pprint import pprint
stuff = [key + ' - ' + 'ICD9s: ' + str(len(apixio_hcc_to_icd[key]['icd9s'])) + ' ICD10s: ' + str(len(apixio_hcc_to_icd[key]['icd10s'])) for key in apixio_hcc_to_icd if key.startswith('V22')]
for key in stuff:
    print key

V22-17 - ICD9s: 18 ICD10s: 21
V22-12 - ICD9s: 115 ICD10s: 261
V22-10 - ICD9s: 326 ICD10s: 517
V22-11 - ICD9s: 127 ICD10s: 134
V22-18 - ICD9s: 44 ICD10s: 180
V22-19 - ICD9s: 7 ICD10s: 6
V22-6 - ICD9s: 18 ICD10s: 37
V22-1 - ICD9s: 3 ICD10s: 3
V22-2 - ICD9s: 33 ICD10s: 49
V22-8 - ICD9s: 46 ICD10s: 75
V22-9 - ICD9s: 97 ICD10s: 113
V22-189 - ICD9s: 15 ICD10s: 291
V22-107 - ICD9s: 39 ICD10s: 103
V22-106 - ICD9s: 4 ICD10s: 128
V22-104 - ICD9s: 21 ICD10s: 119
V22-103 - ICD9s: 15 ICD10s: 45
V22-100 - ICD9s: 11 ICD10s: 76
V22-88 - ICD9s: 3 ICD10s: 27
V22-57 - ICD9s: 60 ICD10s: 12
V22-54 - ICD9s: 21 ICD10s: 108
V22-84 - ICD9s: 18 ICD10s: 27
V22-87 - ICD9s: 24 ICD10s: 13
V22-86 - ICD9s: 12 ICD10s: 18
V22-114 - ICD9s: 15 ICD10s: 14
V22-115 - ICD9s: 27 ICD10s: 32
V22-111 - ICD9s: 15 ICD10s: 14
V22-112 - ICD9s: 56 ICD10s: 91
V22-99 - ICD9s: 6 ICD10s: 35
V22-96 - ICD9s: 7 ICD10s: 14
V22-82 - ICD9s: 10 ICD10s: 11
V22-34 - ICD9s: 1 ICD10s: 2
V22-35 - ICD9s: 13 ICD10s: 77
V22-39 - ICD9s: 175 ICD10s: 626


In [ ]:
import json
code_mappings = json.load(open('/Users/ishekhtman/Documents/Apixio/hoisting table validation/resources/hcc-code-mappings.js'))
print(len(code_mappings))

In [ ]:
for code_mapping in code_mappings:
    if code_mapping['hcc'] == "9" and code_mapping['labelSet'] == "V12":
        print(code_mapping['fromCodeSystem']+":" + code_mapping['fromCode'])

In [ ]:
for code_mapping in code_mappings:
    if code_mapping['fromCode'] == "V12_131":
        print(code_mapping['labelSet']+":"+code_mapping['hcc'])

In [ ]:
for code_mapping in code_mappings:
    from_code = str(code_mapping['fromCode'])
    if 'V12' in from_code:
        to_code = code_mapping['labelSet']+"_" + code_mapping['hcc']
        if from_code != to_code:
            print(code_mapping['fromCode'] + " - " + to_code)